In [1]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
from unicodedata import normalize

import tools as fc

**Configuraciones**


In [2]:
sns.set(style="darkgrid")
pd.set_option('display.float_format', '{:,.2f}'.format)
title_data = 'Materiales extraidos en Colombia'
warnings.filterwarnings("ignore")

**Data**

In [3]:
url_master =  'https://drive.google.com/uc?id=' 
url_minero='https://drive.google.com/file/d/10_YGM_dOKzzLsLNkfB_bN5YeLgDHcrUZ/view?usp=sharing'
url_department = 'https://drive.google.com/file/d/1fKM6yk_oxEsjJyZhOsSjmdcryN7Y50RK/view?usp=sharing'
url_type_miner = 'https://drive.google.com/file/d/1Q69MPSg8joI1nZz6CYwxa0_DdUGquRU2/view?usp=sharing'

url_minero =  url_master + url_minero.split('/')[-2]
url_department =  url_master + url_department.split('/')[-2]
url_type_miner =  url_master + url_type_miner.split('/')[-2]


data_miner = pd.read_csv(url_minero, dtype=str)
data_department = pd.read_csv(url_department, dtype=str)
data_type_miner = pd.read_csv(url_type_miner, dtype=str)

**Data Processing**



must have consistency in the fields jurisdiction in all tables. that's why  they are  convert the values in upper and without accent

In [4]:
data_miner['Municipio Productor'] = data_miner['Municipio Productor'].apply(
    lambda x: normalize('NFKD', x.upper()).encode('ASCII', 'ignore').decode() 
)

data_miner['Departamento'] = data_miner['Departamento'].apply(
    lambda x: normalize('NFKD', x.upper()).encode('ASCII', 'ignore').decode() 
)

data_department[
    'Nombre Departamento'] = data_department['Nombre Departamento'].apply(
    lambda x: normalize('NFKD', x.upper()).encode('ASCII', 'ignore').decode() 
)

data_department[
    'Nombre Municipio'] = data_department['Nombre Municipio'].apply(
    lambda x: normalize('NFKD', x.upper()).encode('ASCII', 'ignore').decode() 
)

Normalization of names of the columns

In [5]:
columns = fc.normalize_name_columns(data_miner.columns)
data_miner.columns = columns

columns = fc.normalize_name_columns(data_department.columns)
data_department.columns = columns

columns = fc.normalize_name_columns(data_type_miner.columns)
data_type_miner.columns = columns

In [13]:
data_miner.shape

(55257, 10)

In [14]:
data_department.shape

(1121, 6)

In [16]:
data_miner

,municipio_productor,departamento,recurso_natural,nombre_del_proyecto,ano_produccion,trimestre,unidad_medida,tipo_contraprestacion,valor_contraprestacion,cantidad_produccion
0,MEDELLIN,ANTIOQUIA,ARCILLAS,PRODUCTORES,2018,Trimestre 4,TONELADAS,REGALIAS,"614,258.00",0.00
1,MEDELLIN,ANTIOQUIA,ARCILLAS CERAMICAS,PRODUCTORES,2014,Trimestre 4,TONELADAS,REGALIAS,"2,036,781.00","14,381.00"
2,MEDELLIN,ANTIOQUIA,ARCILLAS FERRUGINOSAS,PRODUCTORES,2012,Trimestre 1,TONELADAS,REGALIAS,"836,927.00","16,308.00"
3,MEDELLIN,ANTIOQUIA,ARCILLAS MISCELANEAS,PRODUCTORES,2019,Trimestre 1,TONELADAS,REGALIAS,"5,763,839.00","33,348.00"
4,MEDELLIN,ANTIOQUIA,ARCILLAS MISCELANEAS,PRODUCTORES,2022,Trimestre 1,TONELADAS,REGALIAS,"7,187,998.00","28,540.00"
...,...,...,...,...,...,...,...,...,...,...
55252,CUMARIBO,VICHADA,TANTALIO,PRODUCTORES,2018,Trimestre 2,KILOGRAMOS,REGALIAS,"645,095.00",35.00
55253,CUMARIBO,VICHADA,ORO,PRODUCTORES,2019,Trimestre 4,GRAMOS,REGALIAS,"9,251,843.00","1,752.00"
55254,CUMARIBO,VICHADA,NIOBIO,PRODUCTORES,2014,Trimestre 4,KILOGRAMOS,REGALIAS,"23,396,026.00","6,195.00"
55255,CUMARIBO,VICHADA,NIOBIO,PRODUCTORES,2016,Trimestre 4,KILOGRAMOS,REGALIAS,"12,658,396.00","1,442.00"


In [22]:
data_department

,codigo_departamento,codigo_municipio,nombre_departamento,nombre_municipio,region,acceso_al_mar
0,5,5001,ANTIOQUIA,MEDELLIN,Andina,Si
1,5,5002,ANTIOQUIA,ABEJORRAL,Andina,Si
2,5,5004,ANTIOQUIA,ABRIAQUI,Andina,Si
3,5,5021,ANTIOQUIA,ALEJANDRIA,Andina,Si
4,5,5030,ANTIOQUIA,AMAGA,Andina,Si
...,...,...,...,...,...,...
1116,97,97889,VAUPES,YAVARATE,Amazónica,No
1117,99,99001,VICHADA,PUERTO CARRENO,Orinoquía,No
1118,99,99524,VICHADA,LA PRIMAVERA,Orinoquía,No
1119,99,99624,VICHADA,SANTA ROSALIA,Orinoquía,No


In [21]:
pd.merge(data_miner, data_department, on=['departamento', 'municipio_productor'])

KeyError: 'departamento'

In [20]:
data_miner.join(data_department, how='left', on=['departamento', 'municipio_productor'])

ValueError: len(left_on) must equal the number of levels in the index of "right"

In [8]:
data_type_miner.head(2)

,recurso,primera_clasificacion,segunda_clasificacion
0,ARCILLAS,Rocas sedimentarias,NaN
1,ARCILLAS BENTONITICA,Rocas sedimentarias,NaN


change the type of columns, since, all types the features  are object

In [9]:
data_miner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55257 entries, 0 to 55256
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   municipio_productor     55257 non-null  object
 1   departamento            55257 non-null  object
 2   recurso_natural         55257 non-null  object
 3   nombre_del_proyecto     55257 non-null  object
 4   ano_produccion          55257 non-null  object
 5   trimestre               55257 non-null  object
 6   unidad_medida           55257 non-null  object
 7   tipo_contraprestacion   55257 non-null  object
 8   valor_contraprestacion  55257 non-null  object
 9   cantidad_produccion     55257 non-null  object
dtypes: object(10)
memory usage: 4.2+ MB


In [10]:
data_miner[[
    'valor_contraprestacion',
    'cantidad_produccion']] = data_miner[[
    'valor_contraprestacion',
    'cantidad_produccion']].astype('float64')
data_miner[
    'ano_produccion'] = data_miner[
    'ano_produccion'].astype('int64')

In [11]:
asda

NameError: name 'asda' is not defined